In [6]:
# import
import requests
from urllib.parse import urlparse
import urllib.parse
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import re
import seaborn as sns
import json
import folium
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
mpl.rc('font', family='Malgun Gothic')
mpl.rc('axes', unicode_minus=False)
import sqlite3

In [7]:
key_fd = open('./과제data/gov_data_api_key2.txt', mode='r')
govapi_key = key_fd.read(100)
key_fd.close()

In [8]:
start_date = '20200101'
end_date = datetime.today().strftime("%Y%m%d")

### 시도발생현황

In [9]:
corona_url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson'
url = f'{corona_url}?ServiceKey={govapi_key+"3D"}&pageNo=1&numOfRows=10&startCreateDt={start_date}&endCreateDt={end_date}'

In [10]:
result = requests.get(url)
soup = BeautifulSoup(result.text, 'xml')
soup.find('resultCode').get_text()

'00'

In [11]:
url

'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson?ServiceKey=4pmsbWqWKQ1g%2FJklDSCfNZyXKVMH0PLrPbDNJE%2B9%2FFaj9Xc4IqW74iz2dyAVZI8weoJdDn%2Ba%2B0O5fRwJOcF6wg%3D%3D&pageNo=1&numOfRows=10&startCreateDt=20200101&endCreateDt=20201221'

In [12]:
createDt_list,deathCnt_list,defCnt_list,gubun_list,incDec_list = [],[],[],[],[]
isolClearCnt_list,isolIngCnt_list,localOccCnt_list,overFlowCnt_list = [],[],[],[]
qurRate_list,stdDay_list,updateDt_list = [],[],[]
items = soup.find_all('item')
for item in items:
    createDt_list.append(item.find('createDt').string if item.find('createDt') else '')
    deathCnt_list.append(item.find('deathCnt').string if item.find('deathCnt') else '')
    defCnt_list.append(item.find('defCnt').string if item.find('defCnt') else 0)
    gubun_list.append(item.find('gubun').string if item.find('gubun') else '')
    incDec_list.append(item.find('incDec').string if item.find('incDec') else '')
    isolClearCnt_list.append(item.find('isolClearCnt').string if item.find('isolClearCnt') else '')
    isolIngCnt_list.append(item.find('isolIngCnt').string if item.find('isolIngCnt') else '')
    localOccCnt_list.append(item.find('localOccCnt').string if item.find('localOccCnt') else '')
    overFlowCnt_list.append(item.find('overFlowCnt').string if item.find('overFlowCnt') else '')
    qurRate_list.append(item.find('qurRate').string if item.find('qurRate') else '')
    stdDay_list.append(item.find('stdDay').string if item.find('stdDay') else '')
    updateDt_list.append(item.find('updateDt').string if item.find('updateDt') else '')

df = pd.DataFrame({
    '등록시간':createDt_list, '사망자':deathCnt_list, '확진자':defCnt_list,
    '광역시도':gubun_list, '전일대비':incDec_list, '격리해제':isolClearCnt_list, 
    '격리중':isolIngCnt_list, '지역발생':localOccCnt_list,'해외유입':overFlowCnt_list,
    '10만명당':qurRate_list, '기준시간':stdDay_list, '수정시간':updateDt_list
})
df


,등록시간,사망자,확진자,광역시도,전일대비,격리해제,격리중,지역발생,해외유입,10만명당,기준시간,수정시간
0,2020-12-21 09:35:38.526,1,2343,검역,10,1658,684,0,10,-,2020년 12월 21일 00시,null
1,2020-12-21 09:35:38.526,0,252,제주,23,102,150,23,0,18.93,2020년 12월 21일 00시,null
2,2020-12-21 09:35:38.525,3,1077,경남,15,748,326,15,0,26.36,2020년 12월 21일 00시,null
3,2020-12-21 09:35:38.525,61,2035,경북,48,1715,259,48,0,70.05,2020년 12월 21일 00시,null
4,2020-12-21 09:35:38.525,5,508,전남,9,418,85,8,1,25.79,2020년 12월 21일 00시,null
...,...,...,...,...,...,...,...,...,...,...,...,...
5742,2020-03-02 19:27:57.57,0,0,인천,0,,,,,None,2020년 3월 1일 16시,None
5743,2020-03-02 19:27:57.57,9,0,대구,469,,,,,None,2020년 3월 1일 16시,None
5744,2020-03-02 19:27:57.57,0,0,부산,3,,,,,None,2020년 3월 1일 16시,None
5745,2020-03-02 19:27:57.57,0,0,서울,10,,,,,None,2020년 3월 1일 16시,None


In [13]:
df = df[['기준시간', '광역시도', '확진자', '사망자', '전일대비', '격리해제', '격리중', '지역발생', '해외유입', '10만명당']]

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5747 entries, 0 to 5746
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   기준시간    5747 non-null   object
 1   광역시도    5747 non-null   object
 2   확진자     5747 non-null   object
 3   사망자     5747 non-null   object
 4   전일대비    5747 non-null   object
 5   격리해제    5747 non-null   object
 6   격리중     5747 non-null   object
 7   지역발생    5747 non-null   object
 8   해외유입    5747 non-null   object
 9   10만명당   5728 non-null   object
dtypes: object(10)
memory usage: 449.1+ KB


In [15]:
df['일자'] = df['기준시간'].apply(lambda r : r.split('일')[0].replace('년 ','-').replace('월 ','-'))

In [16]:
df['일자']

0       2020-12-21
1       2020-12-21
2       2020-12-21
3       2020-12-21
4       2020-12-21
           ...    
5742      2020-3-1
5743      2020-3-1
5744      2020-3-1
5745      2020-3-1
5746      2020-3-1
Name: 일자, Length: 5747, dtype: object

In [17]:
df['일자'] = pd.to_datetime(df['일자']) 

In [18]:
df['일자2'] = df['일자'].astype(str)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5747 entries, 0 to 5746
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   기준시간    5747 non-null   object        
 1   광역시도    5747 non-null   object        
 2   확진자     5747 non-null   object        
 3   사망자     5747 non-null   object        
 4   전일대비    5747 non-null   object        
 5   격리해제    5747 non-null   object        
 6   격리중     5747 non-null   object        
 7   지역발생    5747 non-null   object        
 8   해외유입    5747 non-null   object        
 9   10만명당   5728 non-null   object        
 10  일자      5747 non-null   datetime64[ns]
 11  일자2     5747 non-null   object        
dtypes: datetime64[ns](1), object(11)
memory usage: 538.9+ KB


In [20]:
for i in df['일자2'].unique():
    if len(df[df['일자2'] == i]) > 20:
        x = df[df['일자2'] == i].tail(len(df[df['일자2'] == i]) - 19).index
        for k in x:
            df.drop(k, inplace=True)

In [21]:
len(df[df['일자2'] == '2020-12-17'])

19

In [22]:
df.head()

,기준시간,광역시도,확진자,사망자,전일대비,격리해제,격리중,지역발생,해외유입,10만명당,일자,일자2
0,2020년 12월 21일 00시,검역,2343,1,10,1658,684,0,10,-,2020-12-21,2020-12-21
1,2020년 12월 21일 00시,제주,252,0,23,102,150,23,0,18.93,2020-12-21,2020-12-21
2,2020년 12월 21일 00시,경남,1077,3,15,748,326,15,0,26.36,2020-12-21,2020-12-21
3,2020년 12월 21일 00시,경북,2035,61,48,1715,259,48,0,70.05,2020-12-21,2020-12-21
4,2020년 12월 21일 00시,전남,508,5,9,418,85,8,1,25.79,2020-12-21,2020-12-21


In [23]:
del df['기준시간']
del df['일자']
df = df[['일자2','광역시도','확진자','사망자','전일대비','격리해제','격리중','지역발생','해외유입','10만명당']]

In [24]:
df.reset_index(drop=True, inplace=True)
df.head()

,일자2,광역시도,확진자,사망자,전일대비,격리해제,격리중,지역발생,해외유입,10만명당
0,2020-12-21,검역,2343,1,10,1658,684,0,10,-
1,2020-12-21,제주,252,0,23,102,150,23,0,18.93
2,2020-12-21,경남,1077,3,15,748,326,15,0,26.36
3,2020-12-21,경북,2035,61,48,1715,259,48,0,70.05
4,2020-12-21,전남,508,5,9,418,85,8,1,25.79


In [25]:
conn = sqlite3.connect('./DB/covid-19.db')
cur = conn.cursor()

In [26]:
sql_table = '''
create table '시·도발생_현황'(
       'rowID' text not null primary key, '일자' text not null,
       '광역시도' text not null,'확진자' int default 0, '사망자' int default 0,
       '전일대비' int default 0, '격리해제' int default 0, '격리중' int default 0,
       '지역발생' int default 0, '해외유입' int default 0, '10만명당' text default '-'
       
)
'''

In [27]:
cur.execute(sql_table)
conn.commit()

In [28]:
sql_insert = 'insert into "시·도발생_현황" values(?,?,?,?,?,?,?,?,?,?,?)'

In [29]:
for i in df.index:
    params = list(df.loc[i])
    params.insert(0, len(df) - i)
    cur.execute(sql_insert, params)
    conn.commit()

In [30]:
conn.close()

In [31]:
df.to_csv('./DB/시도발생현황.csv')

In [32]:
df['확진자'] = df['확진자'].astype(int)

In [33]:
a_df = df.pivot_table('확진자', index=['일자2', '광역시도'])

In [34]:
a_df

확진자
일자2        광역시도         
2020-03-01 강원        0.0
           검역        0.0
           경기        0.0
           경남        0.0
           경북        0.0
...                  ...
2020-12-21 전북      688.0
           제주      252.0
           충남     1288.0
           충북      803.0
           합계    50591.0

[5535 rows x 1 columns]

In [35]:
a_df.to_csv('./DB/광역시도별.csv')

In [36]:
n_df = df.pivot_table('확진자', '일자2', '광역시도')

In [37]:
day_df = n_df - n_df.shift(1)

In [38]:
day_df.fillna(0,inplace=True)

In [39]:
day_df.head()
del day_df['검역']

In [40]:
day_df.astype(int)

광역시도,강원,경기,경남,경북,광주,대구,대전,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북,합계
일자2,,,,,,,,,,,,,,,,,,
2020-03-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2020-03-04,21,101,65,774,13,4006,15,93,99,1,23,9,5,7,3,82,11,5328
2020-03-05,2,9,9,87,1,321,1,-1,4,0,0,0,-1,0,1,4,1,438
2020-03-06,2,10,3,123,-1,367,2,3,2,0,0,0,0,0,0,4,3,518
2020-03-07,1,10,5,65,0,390,0,1,3,1,0,0,0,0,0,2,5,483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-17,9,291,30,9,10,21,11,44,423,0,10,77,0,19,12,19,19,1011
2020-12-18,18,301,44,31,4,20,5,39,398,0,31,65,5,16,27,18,31,1062
2020-12-19,27,281,32,29,9,20,5,39,384,0,9,45,5,15,24,24,103,1055


In [41]:
conn = sqlite3.connect('./DB/covid-19.db')
cur = conn.cursor()

In [42]:
sql_table = '''
create table '일일확진자 현황'(
       '일자' text not null primary key,
       '강원' int not null, '경기' int not null, '경남' int not null,
       '경북' int not null, '광주' int not null, '대구' int not null,
       '대전' int not null, '부산' int not null, '서울' int not null,
       '세종' int not null, '울산' int not null, '인천' int not null,
       '전남' int not null, '전북' int not null, '제주' int not null,
       '충남' int not null, '충북' int not null, '합계' int not null
)
'''

In [43]:
cur.execute(sql_table)
conn.commit()

In [44]:
sql_insert = 'insert into "일일확진자 현황" values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)'

In [45]:
for i in day_df[1:].index:
    params = list(day_df.loc[i])
    params.insert(0, i)
    cur.execute(sql_insert, params)
    conn.commit()

In [46]:
conn.close()

In [47]:
day_df.to_csv('./DB/일일확진자.csv')

### 연령별 현황

In [48]:
corona_url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19GenAgeCaseInfJson?serviceKey='
url = f'{corona_url}{govapi_key+"3D"}&pageNo=1&numOfRows=10&startCreateDt={start_date}&endCreateDt={end_date}'

In [49]:
result = requests.get(url)
soup = BeautifulSoup(result.text, 'xml')
soup.find('resultCode').get_text()

'00'

In [50]:
confCase_list, confCaseRate_list, createDt_list, criticalRate_list = [],[],[],[]
death_list, deathRate_list, gubun_list, id_list, updateDt_list =[],[],[],[],[]
items = soup.find_all('item')
for item in items:
    id_list.append(item.find('seq').string)
    createDt_list.append(item.find('createDt').string.split(' ')[0])
    confCase_list.append(item.find('confCase').string)
    confCaseRate_list.append(item.find('confCaseRate').string)
    criticalRate_list.append(item.find('confCaseRate').string)
    death_list.append(item.find('death').string)
    deathRate_list.append(item.find('deathRate').string)
    gubun_list.append(item.find('gubun').string)
    updateDt_list.append(item.find('updateDt').string if item.find('updateDt') else ' ')

df = pd.DataFrame({
    'rowID':id_list,
    '등록일':createDt_list, 
    '확진자': confCase_list,
    '확진률' : confCaseRate_list,
    '치명률' : criticalRate_list,
    '사망자' : death_list,
    '사망률' : deathRate_list, 
    '구분' : gubun_list,
    '수정일' :  updateDt_list
})
df

,rowID,등록일,확진자,확진률,치명률,사망자,사망률,구분,수정일
0,5565,2020-12-21,1684,3.33,3.33,0,0.00,0-9,null
1,5564,2020-12-21,3150,6.23,6.23,0,0.00,10-19,null
2,5563,2020-12-21,8552,16.9,16.9,0,0.00,20-29,null
3,5562,2020-12-21,6414,12.68,12.68,3,0.43,30-39,null
4,5561,2020-12-21,7099,14.03,14.03,6,0.86,40-49,null
...,...,...,...,...,...,...,...,...,...
2823,5,2020-04-02,1235,12.62,12.62,22,13.58,60-69,None
2824,4,2020-04-02,651,6.65,6.65,46,28.4,70-79,None
2825,3,2020-04-02,442,4.52,4.52,82,50.62,80 이상,None
2826,2,2020-04-02,5881,60.1,60.1,80,49.38,여성,None


In [51]:
df= df[['rowID', '등록일', '구분', '확진자', '확진률', '치명률', '사망자', '사망률', '수정일']]

In [52]:
conn = sqlite3.connect('./DB/covid-19.db')
cur = conn.cursor()

In [53]:
sql_table = '''
create table '연령별·성별감염_현황'(
       'rowID' text not null primary key, '등록일' text not null, '구분' text not null,
       '확진자' int not null, '확진률' int not null, '치명률' float default 0,
       '사망자' int default 0, '사망률' int default 0, 
       '수정일' int default '-'
       )
'''

In [54]:
cur.execute(sql_table)
conn.commit()

In [55]:
len(df.columns)

9

In [56]:
sql_insert = 'insert into "연령별·성별감염_현황" values(?,?,?,?,?,?,?,?,?)'

In [57]:
for i in range(len(df)):
    params = list(df.loc[i])
    cur.execute(sql_insert, params)
    conn.commit()

In [58]:
conn.close()

In [59]:
df.to_csv('./DB/연령별,성별감염_현황.csv')

###  코로나19감염증으로 인한 일별 확진자, 완치자, 치료중인환자, 사망자등에 대한 현황자료

In [60]:
corona_url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson'
url = f'{corona_url}?ServiceKey={govapi_key+"3D"}&pageNo=1&numOfRows=10&startCreateDt={start_date}&endCreateDt={end_date}'

In [61]:
result = requests.get(url)
soup = BeautifulSoup(result.text, 'xml')
soup.find('resultCode').get_text()

'00'

In [62]:
url

'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson?ServiceKey=4pmsbWqWKQ1g%2FJklDSCfNZyXKVMH0PLrPbDNJE%2B9%2FFaj9Xc4IqW74iz2dyAVZI8weoJdDn%2Ba%2B0O5fRwJOcF6wg%3D%3D&pageNo=1&numOfRows=10&startCreateDt=20200101&endCreateDt=20201221'

In [63]:
accDefRate_list, accExamCnt_list, accExamCompCnt_list, careCnt_list = [],[],[],[]
clearCnt_list, deathCnt_list, decideCnt_list, examCnt_list = [],[],[],[]
resutlNegCnt_list, seq_list, stateDt_list, updateDt_list = [],[],[],[]
items = soup.find_all('item')
for item in items:
    seq_list.append(item.find('seq').string)
    stateDt_list.append(item.find('stateDt').string)
    accDefRate_list.append(item.find('accDefRate').string if item.find('accDefRate') else '')
    accExamCnt_list.append(item.find('accExamCnt').string if item.find('accExamCnt') else '')
    accExamCompCnt_list.append(item.find('accExamCompCnt').string if item.find('accExamCompCnt') else '')
    careCnt_list.append(item.find('careCnt').string if item.find('careCnt') else '')
    clearCnt_list.append(item.find('clearCnt').string if item.find('clearCnt') else '')
    deathCnt_list.append(item.find('deathCnt').string if item.find('deathCnt') else '')
    decideCnt_list.append(item.find('decideCnt').string if item.find('decideCnt') else '')
    examCnt_list.append(item.find('examCnt').string if item.find('examCnt') else '')
    resutlNegCnt_list.append(item.find('resutlNegCnt').string if item.find('resutlNegCnt') else '')
    updateDt_list.append(item.find('updateDt').string if item.find('updateDt') else '')

In [64]:
df = pd.DataFrame({
    'rowID': seq_list,
    '기준일': stateDt_list, 
    '누적환진률': accDefRate_list,
    '누적검사수' : accExamCnt_list,
    '누적 검사 완료 수' : accExamCompCnt_list,
    '치료중 환자 수' : careCnt_list,
    '격리해제 수' : clearCnt_list, 
    '사망자 수' : deathCnt_list,
    '확진자 수' :  decideCnt_list,
    '검사 진행 수' : examCnt_list,
    '결과 음성 수 ': resutlNegCnt_list,
    '수정일' : updateDt_list
})
df

,rowID,기준일,누적환진률,누적검사수,누적 검사 완료 수,치료중 환자 수,격리해제 수,사망자 수,확진자 수,검사 진행 수,결과 음성 수,수정일
0,361,20201221,1.4181385274,3713861,3567423,14738,35155,698,50591,146438,3516832,null
1,360,20201220,1.4015332912,3683094,3543619,14269,34722,674,49665,139475,3493954,null
2,359,20201219,1.3798240575,3646247,3520014,13577,34334,659,48570,126233,3471444,null
3,358,20201219,1.3798240575,3646247,3520014,13577,34334,659,48570,126233,3471444,null
4,357,20201218,1.3656965426,3589795,3479177,12888,33982,645,47515,110618,3431662,null
...,...,...,...,...,...,...,...,...,...,...,...,...
358,5,20200205,None,,,,0,0,18,129,,2020-02-05 9:43:16.16
359,4,20200204,None,,,,0,0,0,0,,None
360,3,20200203,None,,,,0,0,15,0,,2020-02-04 14:19:46.46
361,2,20200202,None,,,,2,2,2,2,,None


In [65]:
# 중복자료 지우기
for i in df['rowID'].unique():
    if len(df[df['rowID'] == i]) > 1:
        for k in df[df['rowID'] == i][1:].index:
            df.drop(k, inplace=True)

In [66]:
df.reset_index(drop=True,inplace=True)

In [67]:
conn = sqlite3.connect('./DB/covid-19.db')
cur = conn.cursor()

In [68]:
sql_table = '''
create table '감염_현황'(
    'rowID' text not null primary key, '기준일' text not null,
    '누적환진률' flaot,
    '누적검사수' text, '누적 검사 완료 수' text, '치료중 환자 수' text,
    '격리해제 수' text,   '사망자 수' text , '확진자 수' text,
    '검사 진행 수' text, '결과 음성 수' text, '수정일' text 
    )
'''

In [69]:
cur.execute(sql_table)
conn.commit()

In [70]:
sql_insert = 'insert into "감염_현황" values(?,?,?,?,?,?,?,?,?,?,?,?)'

In [71]:
for i in range(len(df)):
    params = list(df.loc[i])
    cur.execute(sql_insert, params)
    conn.commit()

In [72]:
conn.close()

In [73]:
df.to_csv('./DB/감염현황.csv')